# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# the total number of schools
schools_number = school_data["School ID"].count()

# the total number of students
students_number = student_data["Student ID"].count()

# the total budget
total_budget = school_data["budget"].sum() 

# the average math score
avr_math_score = student_data["math_score"].mean() 

# the average reading score
avr_reading_score = student_data["reading_score"].mean()

# the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_passing_rate = (avr_math_score + avr_reading_score) / 2

# the percentage of students with a passing math score (70 or greater)
percentage_pass_math = (student_data["Student ID"].loc[student_data["math_score"] >= 70].count() / students_number) * 100

# Calculate the percentage of students with a passing reading score (70 or greater)
percentage_pass_reading = (student_data["Student ID"].loc[student_data["reading_score"] >= 70].count() / students_number) * 100

# the Overall Passing Rate
percentage_overall_pass = (percentage_pass_math + percentage_pass_reading) / 2 
avr_overall_pass = (avr_math_score + avr_reading_score)/2

# Create a dataframe to hold the above District Summary results
district_summary_df = pd.DataFrame({"Total Schools": [schools_number],
                                   "Total Students": [students_number],
                                   "Total Budget": [total_budget],
                                   "Average Math Score": [avr_math_score],
                                   "Average Reading Score": [avr_reading_score],
                                   "% Passing Math": [percentage_pass_math],
                                   "% Passing Reading": [percentage_pass_reading],
                                   "Overall Passing Rate": [overall_passing_rate],
                                   "% Overall Passing Rate": [percentage_overall_pass]
                                   }, columns = ["Total Schools",
                                                 "Total Students",
                                                 "Total Budget",
                                                 "Average Math Score",
                                                 "Average Reading Score",
                                                 "% Passing Math",
                                                 "% Passing Reading",
                                                 "Overall Passing Rate",
                                                 "% Overall Passing Rate"
                                                ])

# Improve data formatting 
district_summary_df["Total Students"] = district_summary_df["Total Students"].astype(int).map(
    "{0:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].astype(float).map(
    "${0:,.0f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].astype(float).map(
    "{0:,.6f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].astype(float).map(
    "{0:,.6f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].astype(float).map(
    "{0:,.6f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].astype(float).map(
    "{0:,.6f}%".format)
district_summary_df["Overall Passing Rate"] = district_summary_df["Overall Passing Rate"].astype(float).map(
    "{0:,.6f}".format)
district_summary_df["% Overall Passing Rate"] = district_summary_df["% Overall Passing Rate"].astype(float).map(
    "{0:,.6f}%".format)

district_summary_df   

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.985371,81.877840,74.980853%,85.805463%,80.431606,80.393158%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
# Creat school summary data frame
school_summary_df = school_data.loc[:, ["school_name","type","budget"]] 
school_summary_df.columns = ["School Name","School Type","Total School Budget"]

# Total School Students
total_students = pd.DataFrame(school_data_complete["school_name"].value_counts())
total_students.reset_index(inplace=True)
total_students.columns = ["School Name", "Total Students"]

# Merge school summary data with total students 
school_summary_df = pd.merge(school_summary_df,total_students, on="School Name")

# Per Student Budget
school_summary_df["Per Student Budget"] = school_summary_df["Total School Budget"] / school_summary_df["Total Students"]

# Average Math Score and Average Reading Score
avr_scores = pd.DataFrame(school_data_complete.groupby("school_name")["math_score","reading_score"].mean())
avr_scores.reset_index(inplace=True)
avr_scores.columns = ["School Name","Average Math Score", "Average Reading Score"]

# Merge school summary data with average scores 
school_summary_df = pd.merge(school_summary_df, avr_scores, on="School Name")

# Number Passing Math students
pass_math = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"] >= 70].groupby("school_name")["Student ID"].count())
pass_math.reset_index(inplace=True)
pass_math.columns = ["School Name","Passing Math"]
pass_math

# Merge school summary data with Passing Math students
school_summary_df = pd.merge(school_summary_df, pass_math, on="School Name")

# % Passing Math
school_summary_df["% Passing Math"] = (school_summary_df["Passing Math"] / school_summary_df["Total Students"])*100

# Number Passing Reading students
pass_reading = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"] >= 70].groupby("school_name")["Student ID"].count())
pass_reading.reset_index(inplace=True)
pass_reading.columns = ["School Name","Passing Reading"]
pass_math

# Merge school summary data with Passing Reading students
school_summary_df = pd.merge(school_summary_df, pass_reading, on="School Name")

# % Passing Reading
school_summary_df["% Passing Reading"] = (school_summary_df["Passing Reading"] / school_summary_df["Total Students"])*100

# Overall Passing Rate (Average of the above two)
school_summary_df["% Overall Passing Rate"] = (school_summary_df["% Passing Math"] + school_summary_df["% Passing Reading"])/2

# Remain only required data in the required order
school_summary_df = school_summary_df.loc[:, ["School Name","School Type","Total Students",
                                              "Total School Budget", "Per Student Budget", "Average Math Score",
                                              "Average Reading Score","% Passing Math","% Passing Reading",
                                              "% Overall Passing Rate"]] 
# Save before formatting
school_summary = pd.DataFrame(school_summary_df)

# Set School name as Index
school_summary_df = school_summary_df.set_index("School Name")

best_schools_summary = school_summary_df

# Improve data formatting 
school_summary_df["Total Students"] = school_summary_df["Total Students"].astype(int).map(
   "{0:,}".format)
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].astype(float).map(
   "${0:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].astype(float).map(
   "${0:,.2f}".format)
# school_summary_df["Average Math Score"] = school_summary_df["Average Math Score"].astype(float).map(
#    "{0:,.6f}".format)
# school_summary_df["Average Reading Score"] = school_summary_df["Average Reading Score"].astype(float).map(
#    "{0:,.6f}".format)
# school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"].astype(float).map(
#    "{0:,.6f}%".format)
# school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"].astype(float).map(
#    "{0:,.6f}%".format)
# school_summary_df["% Overall Passing Rate"] = school_summary_df["% Overall Passing Rate"].astype(float).map(
#    "{0:,.6f}%".format)

#school_summary_df["% Overall Passing Rate"].astype(str)

# show school summary information
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,94.860875
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
# Sort data in the discending order
best_schools_summary = school_summary_df.sort_values(["% Overall Passing Rate"], ascending=False)
# Five the best of schools
best_schools_summary.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
# Sort data in the ascending order
werst_schools_summary = school_summary_df.sort_values(["% Overall Passing Rate"])
# Five the werst of schools
werst_schools_summary.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
# Create List of grades
print(list(school_data_complete["grade"].unique()))

['9th', '12th', '11th', '10th']


In [8]:
# select required data
grade_math_df = school_data_complete.loc[:, ["school_name","grade","math_score"]] 

# Create a pandas series for each grade. Hint: use a conditional statement
grade_m_9th = grade_math_df.loc[grade_math_df.grade == "9th"]
grade_m_10th = grade_math_df.loc[grade_math_df.grade == "10th"]
grade_m_11th = grade_math_df.loc[grade_math_df.grade == "11th"]
grade_m_12th = grade_math_df.loc[grade_math_df.grade == "12th"]

# Group each series by school and calculate average value for each grade
grade_m_9th = grade_m_9th.groupby("school_name")["math_score"].mean()
grade_m_10th = grade_m_10th.groupby("school_name")["math_score"].mean()
grade_m_11th = grade_m_11th.groupby("school_name")["math_score"].mean()
grade_m_12th = grade_m_12th.groupby("school_name")["math_score"].mean()

# Combine the series into a dataframe
grade_m_df = pd.DataFrame({"9th": grade_m_9th, 
                           "10th": grade_m_10th, 
                           "11th": grade_m_11th,
                           "12th": grade_m_12th})

# Improve data formatting
grade_m_df["9th"] = grade_m_df["9th"].astype(float).map("{0:,.3f}".format)
grade_m_df["10th"] = grade_m_df["10th"].astype(float).map("{0:,.3f}".format)
grade_m_df["11th"] = grade_m_df["11th"].astype(float).map("{0:,.3f}".format)  
grade_m_df["12th"] = grade_m_df["12th"].astype(float).map("{0:,.3f}".format)

# Math Score by Grade 
grade_m_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.084,76.997,77.516,76.492
Cabrera High School,83.095,83.155,82.766,83.277
Figueroa High School,76.403,76.540,76.884,77.151
Ford High School,77.361,77.672,76.918,76.180
Griffin High School,82.044,84.229,83.842,83.356
Hernandez High School,77.438,77.337,77.136,77.187
Holden High School,83.787,83.430,85.000,82.855
Huang High School,77.027,75.909,76.447,77.226
Johnson High School,77.188,76.691,77.492,76.863


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
# select required data
grade_reading_df = school_data_complete.loc[:, ["school_name","grade","reading_score"]] 

# Create a pandas series for each grade. Hint: use a conditional statement
grade_r_9th = grade_reading_df.loc[grade_reading_df.grade == "9th"]
grade_r_10th = grade_reading_df.loc[grade_reading_df.grade == "10th"]
grade_r_11th = grade_reading_df.loc[grade_reading_df.grade == "11th"]
grade_r_12th = grade_reading_df.loc[grade_reading_df.grade == "12th"]

# Group each series by school and calculate average value for each grade
grade_r_9th = grade_r_9th.groupby("school_name")["reading_score"].mean()
grade_r_10th = grade_r_10th.groupby("school_name")["reading_score"].mean()
grade_r_11th = grade_r_11th.groupby("school_name")["reading_score"].mean()
grade_r_12th = grade_r_12th.groupby("school_name")["reading_score"].mean()

# Combine the series into a dataframe
grade_r_df = pd.DataFrame({"9th": grade_r_9th, 
                           "10th": grade_r_10th, 
                           "11th": grade_r_11th,
                           "12th": grade_r_12th})

# Improve data formatting
grade_r_df["9th"] = grade_r_df["9th"].astype(float).map("{0:,.3f}".format)
grade_r_df["10th"] = grade_r_df["10th"].astype(float).map("{0:,.3f}".format)
grade_r_df["11th"] = grade_r_df["11th"].astype(float).map("{0:,.3f}".format)  
grade_r_df["12th"] = grade_r_df["12th"].astype(float).map("{0:,.3f}".format)

# Reading Score by Grade
grade_r_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303,80.907,80.946,80.912
Cabrera High School,83.676,84.253,83.788,84.288
Figueroa High School,81.199,81.409,80.640,81.385
Ford High School,80.633,81.263,80.404,80.662
Griffin High School,83.369,83.707,84.288,84.014
Hernandez High School,80.867,80.660,81.396,80.857
Holden High School,83.677,83.325,83.816,84.699
Huang High School,81.290,81.512,81.417,80.306
Johnson High School,81.261,80.773,80.616,81.228


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
# create school spending data frame
school_spend_df = pd.DataFrame(school_summary.loc[:, ["Per Student Budget",
                                                      "Average Math Score",
                                                      "Average Reading Score",
                                                      "% Passing Math",
                                                      "% Passing Reading",
                                                      "% Overall Passing Rate"
                                                        ]] )
 
# Figure out the minimum and maximum Spending Ranges (Per Student)
print(school_spend_df["Per Student Budget"].max())
print(school_spend_df["Per Student Budget"].min())

655.0
578.0


In [11]:
# spending bins
spending_bins = [0, 585, 615, 645, 675]

# labels for the bins
group_labels = ["<$585", "$585-615", "$615-645", "$645-675"]

# slice the data and place it into bins and 
# place the data series into a new column inside of the DataFrame
school_spend_df["Spending Ranges (Per Student)"] = pd.cut(school_spend_df["Per Student Budget"], spending_bins, labels=group_labels)
school_spend_df.head()

# Create a Group by "Spending Ranges (Per Student)"
school_spend_df = school_spend_df.groupby("Spending Ranges (Per Student)")

# Calculate the average of each column within the GroupBy object
school_spend_df[["Average Math Score", "Average Reading Score", "% Passing Math",
                 "% Passing Reading", "% Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [12]:
# create school spending data frame
school_size_df = pd.DataFrame(school_summary.loc[:, ["Total Students",
                                                      "Average Math Score",
                                                      "Average Reading Score",
                                                      "% Passing Math",
                                                      "% Passing Reading",
                                                      "% Overall Passing Rate"
                                                        ]] )
 
# Figure out the minimum and maximum Spending Ranges (Per Student)
print(school_size_df["Total Students"].max())
print(school_size_df["Total Students"].min())

4976
427


In [13]:
# size bins
size_bins = [0, 1000, 2000, 5000]

# labels for the bins
group_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# slice the data and place it into bins and 
# place the data series into a new column inside of the DataFrame
school_size_df["School Size"] = pd.cut(school_size_df["Total Students"], size_bins, labels=group_labels)
school_size_df.head()

# Create a Group by "School Size"
school_size_df = school_size_df.groupby("School Size")

# Calculate the average of each column within the GroupBy object
school_size_df[["Average Math Score", "Average Reading Score", "% Passing Math",
                "% Passing Reading", "% Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [14]:
# create school spending data frame
school_type_df = pd.DataFrame(school_summary.loc[:, [ "Average Math Score",
                                                      "Average Reading Score",
                                                      "% Passing Math",
                                                      "% Passing Reading",
                                                      "% Overall Passing Rate",
                                                      "School Type"
                                                        ]] )

# Create a Group by "School Type" and calculate the average of each column within the GroupBy object
school_type_df.groupby("School Type")[["Average Math Score", "Average Reading Score", "% Passing Math",
                 "% Passing Reading", "% Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


## Budget by School Type

In [15]:
# create school spending data frame
school_type_budget = pd.DataFrame(school_summary.loc[:, [ "Total Students",
                                                          "Total School Budget",
                                                          "Per Student Budget",
                                                          "School Type"
                                                        ]] )
#print(school_type_budget)
# Create a Group by "School Type" and calculate the average of each column within the GroupBy object
school_type_budget_df = pd.DataFrame(school_type_budget.groupby("School Type")[["Total Students", "Total School Budget", "Per Student Budget"]].mean())

school_type_budget_df["Total Students"] = round(school_type_budget_df["Total Students"],0)
school_type_budget_df["Total School Budget"] = round(school_type_budget_df["Total School Budget"]/1000000,6)
school_type_budget_df["Per Student Budget"] = round(school_type_budget_df["Per Student Budget"],2)

school_type_budget_df.rename(columns={'Total Students':'Average Students',
                                      'Total School Budget':'Average School Budget(millions)',
                                      'Per Student Budget':'Average Per Student Budget'}, inplace=True)

school_type_budget_df

,Average Students,Average School Budget(millions),Average Per Student Budget
School Type,,,
Charter,1524.0,0.912688,599.50
District,3854.0,2.478275,643.57


# Conclusions Based on PyCity School Analysis

* As a whole,the smaller schools out-performed medium and large size scools across all metrics. 
    
* All 5 the best schools across all metrics are charter schools, despite the fact that two of them have spending budget per student less then 585 and no one belongs to the group with highest spending budget per student (>645).
    
* All 5 the werst schools across all metrics are district schools, despite the fact that two of them have spending budget per student more then 645 and no one belongs to the groups with spending budget per student less then 615.
    
* Despite the fact that the district spends on District Shools in 2.7 times more than budget charter schools since the average budget district schools is 2478275 vs 912688 average budget of charter schools. However the average students number of district schools (3854) in 2.5 times exeeds the average students number (1524) of charter schools which is comparable to their budget.
